# [**Santander Customer Satisfaction**](https://www.kaggle.com/c/santander-customer-satisfaction/)
Which customers are happy customers?  

![](https://storage.googleapis.com/kaggle-competitions/kaggle/4986/media/santander_custsat_red.png)

In [ ]:
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

train_df = pd.read_csv('../input/santander-customer-satisfaction/train.csv')
test_df = pd.read_csv('../input/santander-customer-satisfaction/test.csv')

In [ ]:
plt.figure(figsize=(10, 10))

sns.barplot(
    x=train_df.TARGET.unique(),
    y=train_df.TARGET.value_counts(),
    palette='Pastel1'
)

In [ ]:
sample_0 = np.where(train_df.TARGET==0)[0] # Target == 0 loc
sample_size = train_df.TARGET.value_counts()[1] # Target == 1

In [ ]:
sample_loc = list(np.random.choice(sample_0, sample_size))
target_1_loc = list(np.where(train_df.TARGET==1)[0])
sample_loc.extend(target_1_loc)

In [ ]:
sample_loc.__len__()

In [ ]:
import copy

y_col = 'TARGET'
x_cols = copy.deepcopy(train_df.columns.tolist())
x_cols.remove(y_col)
x_cols.remove('ID')


In [ ]:
x = train_df[x_cols]
y = train_df[y_col]

x_test = test_df[x_cols]

In [ ]:
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
# from sklearn.svm import SVC
from xgboost import XGBClassifier

In [ ]:
x_train, _, y_train, _ = tts(
    x, y,
    train_size=0.7,
    random_state=10
)

ss = StandardScaler()
ss.fit(x_train)

x_train_ss = ss.transform(x_train)
x_test_ss = ss.transform(x_test)

sm = SMOTE(random_state=42)
x_train_ss_smote, y_train_ss_smote = sm.fit_resample(x_train_ss, y_train)

In [ ]:
# svc = SVC()
# svc.fit(x_train_ss_smote, y_train_ss_smote)

xgb = XGBClassifier(
    n_estimators=100,
    learning_rate= 0.05,    
    max_depth=5, 
)
xgb.fit(x_train_ss_smote, y_train_ss_smote)

In [ ]:
# y_pred = svc.predict(x_test_ss)
y_pred = xgb.predict(x_test_ss)

In [ ]:
submission = pd.DataFrame(
    {
        'ID': test_df.ID, 
        'TARGET': y_pred
    }
)
submission.to_csv(r'submission.csv', index=False)